In [1]:
import keras
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
import numpy as np
import pandas as pd
import glob

#read in data
df = pd.read_excel('data/fer2013.xlsx')

Using TensorFlow backend.


In [2]:
from math import ceil 

#set up train/test indices
indices = list(range(len(df[df['Usage'] == 'Training'])))
np.random.shuffle(indices)
train_indices = indices[ceil(len(indices)/5):]
valid_indices = indices[:ceil(len(indices)/5)]
labels = df.emotion

In [3]:
#split train/test/validation sets, and do one hot encoding on labels
one_hot_labels = keras.utils.to_categorical(df.emotion, num_classes=len(df.emotion.unique()))
test_labels = one_hot_labels[~df.index.isin(indices)]
train_labels = one_hot_labels[train_indices]
valid_labels = one_hot_labels[valid_indices]

In [4]:
#converts gray scale images to RGB to make it appropriate for keras preprocessing tools
def to_rgb1a(im):
    # Convert to RGB
    w, h = im.shape
    ret = np.empty((w, h, 3), dtype=np.float16)
    ret[:, :, 2] =  ret[:, :, 1] =  ret[:, :, 0] =  im
    return ret

In [5]:
%%time
#convert list of pixel values to RGB arrays
df['pixels'] = df['pixels'].apply(lambda x: to_rgb1a(np.reshape(np.array(x.split()),(48,48))))

Wall time: 1min 56s


In [21]:
len(df)

35887

In [22]:
df.head()

,emotion,pixels,Usage
0,0,"[[[70.0, 70.0, 70.0], [80.0, 80.0, 80.0], [82....",Training
1,0,"[[[151.0, 151.0, 151.0], [150.0, 150.0, 150.0]...",Training
2,2,"[[[231.0, 231.0, 231.0], [212.0, 212.0, 212.0]...",Training
3,4,"[[[24.0, 24.0, 24.0], [32.0, 32.0, 32.0], [36....",Training
4,6,"[[[4.0, 4.0, 4.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",Training


In [6]:
#function to add a dimension and vertically stack arrays
def paths_to_tensor(arrs):
    list_of_tensors = [np.expand_dims(arr, axis = 0) for arr in arrs]
    return np.vstack(list_of_tensors)

In [7]:
img_input = preprocess_input(paths_to_tensor(df['pixels']))

In [152]:
from keras.applications.vgg19 import VGG19
#initialize VGG19 model
VGG19model = VGG19(include_top = False, weights = 'imagenet')
VGG19model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [153]:
import keras
from keras.applications.vgg16 import VGG16
#initialize VGG16 model
VGG16model = VGG16(include_top = False, weights = 'imagenet')
VGG16model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [130]:
%%time

#save bottleneck features for training data
VGG16_bottleneck_features_train = VGG16model.predict(img_input[train_indices])
np.save('VGG16_bottleneck_features_train.npy', VGG16_bottleneck_features_train) 

VGG19_bottleneck_features_train = VGG19model.predict(img_input[train_indices])
np.save('VGG19_bottleneck_features_train.npy', VGG19_bottleneck_features_train) 

Wall time: 24min 30s


In [136]:
%%time

#save VGG16 bottleneck features for validation data
VGG16_bottleneck_features_valid = VGG16model.predict(img_input[valid_indices])
np.save('VGG16_bottleneck_features_valid.npy', VGG16_bottleneck_features_valid) 

#load features
VGG16_train_data  = np.load('VGG16_bottleneck_features_train.npy')
VGG16_valid_data = np.load('VGG16_bottleneck_features_valid.npy')

Wall time: 3min 2s


In [137]:
#save VGG19 bottleneck features for validation data
VGG19_bottleneck_features_valid = VGG19model.predict(img_input[valid_indices])
np.save('VGG19_bottleneck_features_valid.npy', VGG19_bottleneck_features_valid) 

#load features
VGG19_train_data  = np.load('VGG19_bottleneck_features_train.npy')
VGG19_valid_data = np.load('VGG19_bottleneck_features_valid.npy')

In [139]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

#define model architecture
model = Sequential()
model.add(Flatten(input_shape=VGG19_train_data.shape[1:]))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 7)                 3591      
Total params: 266,247
Trainable params: 266,247
Non-trainable params: 0
_________________________________________________________________


In [149]:
from keras.callbacks import ModelCheckpoint

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='C:/Users/Alvin/.keras/models/weights.best.VGG19.face-{epoch:02d}-{val_loss:.2f}.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(VGG19_train_data, train_labels,
          epochs=20,
          batch_size=20,
          validation_data=(VGG19_valid_data, valid_labels), callbacks = [checkpointer])
model.save_weights('bottleneck_fc_model.h5')

Train on 22967 samples, validate on 5742 samples
Epoch 1/20
22967/22967 [==============================] - 12s - loss: 10.1111 - acc: 0.3622 - val_loss: 9.8094 - val_acc: 0.3797
Epoch 2/20
22967/22967 [==============================] - 10s - loss: 10.0018 - acc: 0.3679 - val_loss: 9.9208 - val_acc: 0.3758
Epoch 3/20
22967/22967 [==============================] - 10s - loss: 9.9870 - acc: 0.3705 - val_loss: 9.8016 - val_acc: 0.3812
Epoch 4/20
22967/22967 [==============================] - 12s - loss: 10.0138 - acc: 0.3682 - val_loss: 9.8433 - val_acc: 0.3802
Epoch 5/20
22967/22967 [==============================] - 11s - loss: 10.0247 - acc: 0.3680 - val_loss: 9.9097 - val_acc: 0.3765
Epoch 6/20
22967/22967 [==============================] - 11s - loss: 10.0090 - acc: 0.3688 - val_loss: 9.9064 - val_acc: 0.3751
Epoch 7/20
22967/22967 [==============================] - 11s - loss: 9.9614 - acc: 0.3725 - val_loss: 9.8524 - val_acc: 0.3786
Epoch 8/20
22967/22967 [==========================